## 자전거 대여 수요 예측
#### 2011년 1월부터 2012년 12월까지 날짜/시간, 기온, 습도, 풍속 등의 정보를 기반으로 1시간 간격 동안의 자전거 대여 횟수가 기재되어 있다.

- datetime: hourly date + timestamp
- season : 1=봄, 2=여름, 3=가을, 4=겨울
- holiday : 1=토,일요일의 주말을 제외한 국경일 등의 휴일, 0=휴일이 아닌 날
- workingday : 1=토,일요일의 주말 및 휴일이 아닌 주중, 0=주말 및 휴일
- weather:
 - 1=맑음, 약간 구름 낀 흐림
 - 2=안개, 안개+흐림
 - 3=가벼운 눈, 가벼운 비+천둥
 - 4=심한 눈/비, 천둥/번개
- temp : 온도 (섭씨)
- atemp : 체감온도 (섭씨)
- humidity : 상대습도
- windspeed : 풍속
- casual : 사전에 등록되지 않은 사용자가 대여한 횟수
- registered : 사전에 등록된 사용자가 대여한 횟수
- count : 대여 횟수

In [30]:
import pandas as pd
import numpy as np
import seaborn as sns

In [31]:
import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)

# 1. 데이터 불러오기 및 전처리
### (1) bikeshare.csv를 pandas를 이용해 bike_df의 이름으로 저장하고 행렬 개수와 5개의 자료를 뽑아보시오.

In [32]:
bike_df = pd.read_csv("bikeshare.csv")
bike_df.shape
print(bike_df.head())

              datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2  2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3  2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4  2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  


## (2) bike_df에서 항목별 데이터타입을 확인한는 코드를 작성하시오.

In [33]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


### (3) pandas를 이용해 datetime의 칼럼을 년, 월, 일, 시간과 같은 4개의 속성으로 분리하시오.
- 판다스에서 문자열을 datatime 타입으로 변환하는 apply(pd.to_datetime) 메서드를 이용해 datetime이라는 이름의 새로운 칼럼을 만드시오.

In [34]:
bike_df['datetime'] = bike_df['datetime'].apply(pd.to_datetime)

In [35]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.7 KB


In [36]:
bike_df['year'] = bike_df['datetime'].dt.year
bike_df['month'] = bike_df['datetime'].dt.month
bike_df['day'] = bike_df['datetime'].dt.day
bike_df['time'] = bike_df['datetime'].dt.time

bike_df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,time
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,00:00:00
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,01:00:00
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,02:00:00
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,1,03:00:00
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,1,04:00:00


In [37]:
bike_df.drop(['datetime','casual','registered'], axis = 1,inplace= True)

# 2. 선형회귀 실습하기

### (1) X_fearues와 y_target이라는 이름으로 독립변수와 종속변수를 분리하시오. (종속변수 = count)

In [38]:
X_features = bike_df.drop(['count'],axis=1)
y_target = bike_df['count']

In [40]:
X_features.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,time
0,1,0,0,1,9.84,14.395,81,0.0,2011,1,1,00:00:00
1,1,0,0,1,9.02,13.635,80,0.0,2011,1,1,01:00:00
2,1,0,0,1,9.02,13.635,80,0.0,2011,1,1,02:00:00
3,1,0,0,1,9.84,14.395,75,0.0,2011,1,1,03:00:00
4,1,0,0,1,9.84,14.395,75,0.0,2011,1,1,04:00:00


### (2) 숫자형 카테고리 값을 pandas의 get_dummies를 이용해 원-핫 인코딩하시오.
- 현재 데이터타입이 숫자형 카테고리인 칼럼은 year,month,day,hour,holiday,workingday,season,weather 입니다.
- X_features_ohe란 이름에 저장하세요.

In [47]:
X_features_ohe = pd.get_dummies(bike_df, columns = ['year','month','day','time','holiday','workingday','season','weather'])
X_features_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   temp           10886 non-null  float64
 1   atemp          10886 non-null  float64
 2   humidity       10886 non-null  int64  
 3   windspeed      10886 non-null  float64
 4   count          10886 non-null  int64  
 5   year_2011      10886 non-null  uint8  
 6   year_2012      10886 non-null  uint8  
 7   month_1        10886 non-null  uint8  
 8   month_2        10886 non-null  uint8  
 9   month_3        10886 non-null  uint8  
 10  month_4        10886 non-null  uint8  
 11  month_5        10886 non-null  uint8  
 12  month_6        10886 non-null  uint8  
 13  month_7        10886 non-null  uint8  
 14  month_8        10886 non-null  uint8  
 15  month_9        10886 non-null  uint8  
 16  month_10       10886 non-null  uint8  
 17  month_11       10886 non-null  uint8  
 18  month_

### (3) 사이킷런에서 train_test_split을 이용해 훈련세트와 테스트 세트를 분리하시오.
- 이때 종속변수는 count 칼럼이고 독립변수는 종속변수 이외의 모든 칼럼입니다.
- X_train,X_test,y_train,y_test 이란 이름으로 세트를 만들고 test_size=0.3,random_state=0 이라는 옵션 추가하세요.

In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features_ohe, y_target,test_size = 0.3,random_state = 0)

# (4) 회귀 종류별로 실습
#### i. 선형 회귀
- 사이킷런에서 LinearRegression 을 불러와 fit한 후 X_test를 예측하여 pred란 이름에 저장하시오.
- 사이킷런에서의 mean_square_error과 np.sqrt()를 이용해 y_test와 pred의 RMSE를 출력하시오.
- 이 모델의 intercept와 coefficient를 출력하시오.

In [52]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)

pred = lin_reg.predict(X_test)

In [55]:
np.sqrt(mean_squared_error(y_test,pred))

1.0105281896952956e-13

In [56]:
lin_reg.intercept_, lin_reg.coef_

(-0.0010611810342879835,
 array([-1.72755142e-14,  2.34257058e-14, -9.85322934e-16,  5.19029264e-15,
         1.00000000e+00,  1.17406695e-04,  1.17406695e-04,  4.49228070e-04,
         4.49228070e-04,  4.49228070e-04,  1.86026635e-05,  1.86026635e-05,
         1.86026635e-05, -4.36972744e-04, -4.36972744e-04, -4.36972744e-04,
         3.28768915e-04,  3.28768915e-04,  3.28768915e-04,  1.21253929e-05,
         1.21253929e-05,  1.21253929e-05,  1.21253929e-05,  1.21253929e-05,
         1.21253929e-05,  1.21253929e-05,  1.21253929e-05,  1.21253929e-05,
         1.21253929e-05,  1.21253929e-05,  1.21253929e-05,  1.21253929e-05,
         1.21253929e-05,  1.21253929e-05,  1.21253929e-05,  1.21253929e-05,
         1.21253929e-05,  1.21253929e-05,  5.60086331e-07,  5.60086330e-07,
         5.60086330e-07,  5.60086331e-07,  5.60086330e-07,  5.60086330e-07,
         5.60086332e-07,  5.60086331e-07,  5.60086331e-07,  5.60086328e-07,
         5.60086332e-07,  5.60086332e-07,  5.60086328e-07,  5.6

#### ii. 확률적 경사 하강법을 이용한 선형 회귀
- 사이킷런에서 제공하는 SGDRegressor을 이용해 위와 같은 과정을 반복하시오.
- 이때 max_iter=1000,tol=1e-3,penalty=None,eta0=0.1 인 옵션을 추가하시오.

In [60]:
from sklearn.linear_model import SGDRegressor
sgd_reg= SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1)
sgd_reg.fit(X_train,y_train)

pred_sgd = sgd_reg.predict(X_test)

In [61]:
np.sqrt(mean_squared_error(y_test,pred_sgd))

421918459498150.2

In [63]:
sgd_reg.intercept_, sgd_reg.coef_

(array([5.53791975e+11]),
 array([-2.03813267e+10, -5.53607485e+09, -2.69807896e+11, -3.28997548e+11,
         1.63924844e+12,  7.04042637e+11, -1.50250662e+11, -4.39248825e+10,
         4.56062374e+11, -2.95668734e+11,  3.84940337e+11, -3.61390339e+11,
         7.49429783e+11, -8.81012061e+10, -2.30200248e+11,  1.91856174e+11,
         3.68560981e+09, -1.49034288e+11, -6.38626045e+10,  2.74303670e+11,
         1.28092047e+11,  2.57485790e+11,  9.90232691e+11,  1.65209567e+11,
        -2.21956303e+10, -7.02221780e+10, -2.09519731e+11,  1.95867229e+09,
        -7.81035013e+10, -4.62042791e+11,  1.42158949e+11,  2.30366520e+11,
        -5.80891774e+11, -1.13331126e+12,  6.04503114e+11, -9.52860451e+10,
         2.44314586e+10,  3.86622402e+11, -7.33357276e+10,  4.22754693e+11,
         4.55049709e+10, -1.35621305e+11, -1.77653156e+11,  4.19990092e+10,
        -8.44813087e+09,  2.28681132e+11, -2.96862401e+11, -3.06189375e+11,
         7.31085569e+11,  1.15391941e+12, -2.04002228e+11, -7.